In [3]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

%matplotlib inline

In [7]:
june_trip = pd.read_csv('../data/june_trip.csv')

In [8]:
june_trip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205627 entries, 0 to 205626
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   pubTimeStamp    205627 non-null  object 
 1   companyName     205627 non-null  object 
 2   tripRecordNum   205627 non-null  object 
 3   sumdID          205627 non-null  object 
 4   tripDuration    205627 non-null  float64
 5   tripDistance    205627 non-null  float64
 6   startDate       205627 non-null  object 
 7   startTime       205627 non-null  object 
 8   endDate         205627 non-null  object 
 9   endTime         205627 non-null  object 
 10  startLatitude   205627 non-null  float64
 11  startLongitude  205627 non-null  float64
 12  endLatitude     205627 non-null  float64
 13  endLongitude    205627 non-null  float64
 14  tripRoute       205627 non-null  object 
 15  create_dt       205627 non-null  object 
dtypes: float64(6), object(10)
memory usage: 25.1+ MB


In [11]:
june_trip['start_dt'] = pd.to_datetime(pd.to_datetime(june_trip['startDate']).astype(str) + ' ' + june_trip['startTime'])
june_trip['end_dt'] = pd.to_datetime(pd.to_datetime(june_trip['endDate']).astype(str) + ' ' + june_trip['endTime'])
june_trip['time_diff'] = (june_trip.end_dt - june_trip.start_dt )
june_trip.head()

,pubTimeStamp,companyName,tripRecordNum,sumdID,tripDuration,tripDistance,startDate,startTime,endDate,endTime,startLatitude,startLongitude,endLatitude,endLongitude,tripRoute,create_dt,start_dt,end_dt,time_diff
0,2019-06-01 00:05:46.817000,Bird,BRD3572,PoweredSPI1T,4.0,328.084000,2019-06-01 00:00:00,00:02:18.203333,2019-06-01 00:00:00,00:06:16.406666,36.164400,-86.78070,36.163600,-86.780200,"[[36.164679,-86.781089],[36.163693,-86.78011],...",2019-06-02 05:30:19.960000,2019-06-01 00:02:18.203333,2019-06-01 00:06:16.406666,0 days 00:03:58.203333
1,2019-06-01 00:05:46.817000,Bird,BRD3571,Powered2I3MS,5.0,4921.260000,2019-06-01 00:00:00,00:02:44.803333,2019-06-01 00:00:00,00:07:28.286666,36.175300,-86.79430,36.175300,-86.794300,"[[36.175367,-86.794232],[36.175367,-86.794232]...",2019-06-02 05:30:19.927000,2019-06-01 00:02:44.803333,2019-06-01 00:07:28.286666,0 days 00:04:43.483333
2,2019-06-01 00:09:54,Gotcha,GOT1,Powered327,12.0,12.426575,2019-06-01 00:00:00,00:09:56,2019-06-01 00:00:00,00:21:56,36.161501,-86.77601,36.152529,-86.783742,"[[""36.16149"",""-86.77605000000001""]]",2019-06-06 22:23:08.673000,2019-06-01 00:09:56.000000,2019-06-01 00:21:56.000000,0 days 00:12:00
3,2019-06-01 00:10:46.957000,Bird,BRD3610,Powered8U1A6,2.0,0.000000,2019-06-01 00:00:00,00:10:31.163333,2019-06-01 00:00:00,00:12:02.773333,36.164000,-86.78070,36.163100,-86.779700,"[[36.163168,-86.779639]]",2019-06-02 05:30:20.283000,2019-06-01 00:10:31.163333,2019-06-01 00:12:02.773333,0 days 00:01:31.610000
4,2019-06-01 00:10:46.957000,Bird,BRD3612,PoweredSPI1T,5.0,656.168000,2019-06-01 00:00:00,00:07:21.430000,2019-06-01 00:00:00,00:12:30.913333,36.165000,-86.77990,36.165900,-86.777800,"[[36.164951,-86.779836],[36.16494,-86.779456],...",2019-06-02 05:30:20.347000,2019-06-01 00:07:21.430000,2019-06-01 00:12:30.913333,0 days 00:05:09.483333


In [12]:
june_trip.companyName.value_counts()

Lime             84728
Bird             55014
Lyft             37821
SPIN             14294
Bolt Mobility    10219
Gotcha            1803
JUMP              1748
Name: companyName, dtype: int64

In [14]:
lime = june_trip.loc[(june_trip['companyName'] == 'Lime')]
lyft = june_trip.loc[(june_trip['companyName'] == 'Lyft')]
bird = june_trip.loc[(june_trip['companyName'] == 'Bird')]
spin = june_trip.loc[(june_trip['companyName'] == 'SPIN')]
bolt = june_trip.loc[(june_trip['companyName'] == 'Bolt Mobility')]
gotcha = june_trip.loc[(june_trip['companyName'] == 'Gotcha')]
jump = june_trip.loc[(june_trip['companyName'] == 'JUMP')]

In [15]:
lime.shape

(84728, 19)

In [16]:
lyft.head()

,pubTimeStamp,companyName,tripRecordNum,sumdID,tripDuration,tripDistance,startDate,startTime,endDate,endTime,startLatitude,startLongitude,endLatitude,endLongitude,tripRoute,create_dt,start_dt,end_dt,time_diff
275,2019-06-01 00:54:40.193000,Lyft,LFT114,Powered878687,2.072250,580.70868,2019-06-01 00:00:00,00:52:35.763333,2019-06-01 00:00:00,00:54:40.100000,36.16146,-86.77741,36.16273,-86.77763,"[[36.16146,-86.77741],[36.16151,-86.77743],[36...",2019-06-02 08:34:39.950000,2019-06-01 00:52:35.763333,2019-06-01 00:54:40.100000,0 days 00:02:04.336667
276,2019-06-01 00:54:43.317000,Lyft,LFT115,Powered449699,7.980650,4012.46732,2019-06-01 00:00:00,00:46:44.376666,2019-06-01 00:00:00,00:54:43.216666,36.17856,-86.79096,36.17875,-86.79093,"[[36.17856,-86.79096],[36.17862,-86.79099],[36...",2019-06-02 08:34:39.983000,2019-06-01 00:46:44.376666,2019-06-01 00:54:43.216666,0 days 00:07:58.840000
362,2019-06-01 01:11:17.377000,Lyft,LFT158,Powered033388,24.994950,16049.86928,2019-06-01 00:00:00,00:46:17.583333,2019-06-01 00:00:00,01:11:17.280000,36.17639,-86.75218,36.19534,-86.72350,"[[36.17639,-86.75218],[36.17641,-86.75223],[36...",2019-06-02 08:34:40.060000,2019-06-01 00:46:17.583333,2019-06-01 01:11:17.280000,0 days 00:24:59.696667
373,2019-06-01 01:12:30.443000,Lyft,LFT159,Powered878385,1.521600,52.49344,2019-06-01 00:00:00,01:10:58.910000,2019-06-01 00:00:00,01:12:30.206666,36.15256,-86.79106,36.15261,-86.79112,"[[36.15256,-86.79106],[36.1526,-86.79109],[36....",2019-06-02 08:34:40.090000,2019-06-01 01:10:58.910000,2019-06-01 01:12:30.206666,0 days 00:01:31.296666
431,2019-06-01 01:22:30.543000,Lyft,LFT184,Powered689177,2.386933,0.00000,2019-06-01 00:00:00,01:20:07.213333,2019-06-01 00:00:00,01:22:30.426666,36.14656,-86.80973,36.14656,-86.80975,"[[36.14656,-86.80973],[36.14656,-86.80975]]",2019-06-02 08:34:40.127000,2019-06-01 01:20:07.213333,2019-06-01 01:22:30.426666,0 days 00:02:23.213333


In [17]:
jump.head(10)

,pubTimeStamp,companyName,tripRecordNum,sumdID,tripDuration,tripDistance,startDate,startTime,endDate,endTime,startLatitude,startLongitude,endLatitude,endLongitude,tripRoute,create_dt,start_dt,end_dt,time_diff
81384,2019-06-10 18:56:13,JUMP,JMP2,Poweredb671ffe4-2bca-5880-98ec-d36f65aa11a2,3.228833,528.0,2019-06-10 00:00:00,13:55:46,2019-06-10 00:00:00,13:59:00,36.154133,-86.78408,36.154457,-86.78590,"[[""36.154133"",""-86.784078""],[""36.154133"",""-86....",2019-06-11 07:12:12.610000,2019-06-10 13:55:46,2019-06-10 13:59:00,0 days 00:03:14
85029,2019-06-11 07:50:56,JUMP,JMP20,Poweredb671ffe4-2bca-5880-98ec-d36f65aa11a2,6.876867,1900.8,2019-06-11 00:00:00,01:50:35,2019-06-11 00:00:00,01:57:28,36.161880,-86.77778,36.163807,-86.77894,"[[""36.16188"",""-86.777775""],[""36.16188"",""-86.77...",2019-06-12 07:14:00.957000,2019-06-11 01:50:35,2019-06-11 01:57:28,0 days 00:06:53
85030,2019-06-11 07:50:56,JUMP,JMP21,Poweredb671ffe4-2bca-5880-98ec-d36f65aa11a2,9.020367,2904.0,2019-06-11 00:00:00,01:52:08,2019-06-11 00:00:00,02:01:10,36.162930,-86.77624,36.157124,-86.78098,"[[""36.162928"",""-86.776241""],[""36.162928"",""-86....",2019-06-12 07:14:00.993000,2019-06-11 01:52:08,2019-06-11 02:01:10,0 days 00:09:02
85031,2019-06-11 07:50:56,JUMP,JMP22,Poweredb671ffe4-2bca-5880-98ec-d36f65aa11a2,3.628050,686.4,2019-06-11 00:00:00,02:03:56,2019-06-11 00:00:00,02:07:34,36.175200,-86.78806,36.174450,-86.78642,"[[""36.1752"",""-86.78806""],[""36.175201"",""-86.788...",2019-06-12 07:14:01.027000,2019-06-11 02:03:56,2019-06-11 02:07:34,0 days 00:03:38
85032,2019-06-11 07:50:56,JUMP,JMP23,Poweredb671ffe4-2bca-5880-98ec-d36f65aa11a2,24.942583,5808.0,2019-06-11 00:00:00,02:01:26,2019-06-11 00:00:00,02:26:23,36.146250,-86.79947,36.147650,-86.80016,"[[""36.14623"",""-86.799606""],[""36.146001"",""-86.7...",2019-06-12 07:14:01.060000,2019-06-11 02:01:26,2019-06-11 02:26:23,0 days 00:24:57
85033,2019-06-11 07:50:56,JUMP,JMP27,Poweredb671ffe4-2bca-5880-98ec-d36f65aa11a2,13.601434,7867.2,2019-06-11 00:00:00,02:39:45,2019-06-11 00:00:00,02:53:21,36.164104,-86.77997,36.176693,-86.77633,"[[""36.164103"",""-86.779968""],[""36.164103"",""-86....",2019-06-12 07:14:01.200000,2019-06-11 02:39:45,2019-06-11 02:53:21,0 days 00:13:36
85034,2019-06-11 07:50:56,JUMP,JMP29,Poweredb671ffe4-2bca-5880-98ec-d36f65aa11a2,2.418783,422.4,2019-06-11 00:00:00,03:14:59,2019-06-11 00:00:00,03:17:24,36.161650,-86.77789,36.162710,-86.77732,"[[""36.161651"",""-86.77789""],[""36.16271"",""-86.77...",2019-06-12 07:14:01.267000,2019-06-11 03:14:59,2019-06-11 03:17:24,0 days 00:02:25
85035,2019-06-11 07:50:56,JUMP,JMP28,Poweredb671ffe4-2bca-5880-98ec-d36f65aa11a2,18.223516,6072.0,2019-06-11 00:00:00,02:49:02,2019-06-11 00:00:00,03:07:15,36.153767,-86.79795,36.158924,-86.78230,"[[""36.154358"",""-86.798495""],[""36.154403"",""-86....",2019-06-12 07:14:01.233000,2019-06-11 02:49:02,2019-06-11 03:07:15,0 days 00:18:13
85036,2019-06-11 07:50:56,JUMP,JMP24,Poweredb671ffe4-2bca-5880-98ec-d36f65aa11a2,4.759483,0.0,2019-06-11 00:00:00,02:34:38,2019-06-11 00:00:00,02:39:23,36.158356,-86.77517,36.158356,-86.77517,"[[""36.158355"",""-86.77517""],[""36.158355"",""-86.7...",2019-06-12 07:14:01.097000,2019-06-11 02:34:38,2019-06-11 02:39:23,0 days 00:04:45
85037,2019-06-11 07:50:56,JUMP,JMP26,Poweredb671ffe4-2bca-5880-98ec-d36f65aa11a2,5.008383,2745.6,2019-06-11 00:00:00,02:36:57,2019-06-11 00:00:00,02:41:57,36.177307,-86.79276,36.174625,-86.78579,"[[""36.17731"",""-86.79268""],[""36.177341"",""-86.79...",2019-06-12 07:14:01.130000,2019-06-11 02:36:57,2019-06-11 02:41:57,0 days 00:05:00


In [21]:
lime_time_1 = lime.time_diff/pd.Timedelta(hours=1)
lime_time_1.sum()

20785.105277777777

In [22]:
lime_5 = lime.time_diff/pd.Timedelta(minutes=1)
round(lime_5.mean(),3)

14.719

In [23]:
lyft_5 = lyft.time_diff/pd.Timedelta(minutes=1)
round(lyft_5.mean(),3)

15.869

In [24]:
bird_5 = bird.time_diff/pd.Timedelta(minutes=1)
round(bird_5.mean(),3)

13.4

In [25]:
spin_5 = spin.time_diff/pd.Timedelta(minutes=1)
round(spin_5.mean(),3)

24.718

In [26]:
bolt_5 = bolt.time_diff/pd.Timedelta(minutes=1)
round(bolt_5.mean(),3)

22.355

In [27]:
gotcha_5 = gotcha.time_diff/pd.Timedelta(minutes=1)
round(gotcha_5.mean(),3)

10.452

In [28]:
jump_5 = jump.time_diff/pd.Timedelta(minutes=1)
round(jump_5.mean(),3)

56.172